# **Tarea - Redes Neuronales Artificiales**
###  MLP - Clasificación
* _Maria Alejandra Bonilla Diaz - 20251595002_ 
* _Alvaro Alejandro Zarabanda Gutierrez – 20251595006_
* _Youssef Alejandro Ortiz Vargas – 20251595004_

En el procesamiento de bebidas, la aplicación de un determinado conservante es realizada en función de la combinación de 4 variables reales, definidas por:

- $x_1$ (cantidad de agua)
- $x_2$ (grado de acidez)
- $x_3$ (temperatura)
- $x_4$ (tensión superficial)

Se sabe que solo existen tres tipos de conservantes que pueden ser aplicados, los cuales son categorizados como A, B y C. A partir de estas variables, se realizan pruebas de laboratorio para determinar qué tipo de conservante debe ser aplicado en determinada bebida.

De 148 pruebas experimentales, un equipo de expertos decide aplicar una red MLP como clasificador de patrones, con el fin de que esta identifique qué conservante será aplicado en un determinado lote de bebida. Por las características de la línea de producción, se utilizará una red con tres salidas, como se muestra en la figura.

![Enunciado](./assets/enunciado_tMLP-Clasificacion.PNG)

De tal manera, la salida de la red representa el conservante a ser aplicado y está definida por:

| Tipo de Conservante | y₁ | y₂ | y₃ |
|---------------------|----|----|----|
| Tipo A              | 1  | 0  | 0  |
| Tipo B              | 0  | 1  | 0  |
| Tipo C              | 0  | 0  | 1  |

Utilizando los datos de entrenamiento presentados en el Anexo, ejecute el entrenamiento de una red MLP (4 entradas, 15 neuronas en la capa escondida y 3 salidas) que pueda clasificar, en función de los valores medidos $x_1$, $x_2$, $x_3$ y $x_4$ (ya normalizados), qué tipo de conservante debe ser aplicado en determinada bebida. De acuerdo con lo anterior, realice las siguientes actividades:


## Punto 1:
Ejecutar cinco entrenamientos para la red MLP mostrada en la figura, por medio del algoritmo de aprendizaje **backpropagation**, inicializando todas las matrices de pesos con valores aleatorios entre 0 y 1. Utilice la función de activación logística para todos los neurones, tasa de aprendizaje $\eta = 0.1$ y precisión $\varepsilon = 10^{-6}$. Mida también el tiempo de procesamiento involucrado en cada uno de estos entrenamientos.


In [1]:
import numpy as np
import pandas as pd
import time

# 1. Cargar datos limpios
df = pd.read_csv("assets/tareaClasificacion_datos.csv")

X = df[["x1", "x2", "x3", "x4"]].values
Y = df[["d1", "d2", "d3"]].values

# 2. Funciones del MLP
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    return x * (1 - x)

# 3. Entrenar una red
def entrenar_MLP(X, Y, eta=0.1, epsilon=1e-3, max_epochs=100000):

    n_inputs = 4
    n_hidden = 15
    n_outputs = 3

    # PESOS INICIALES ALEATORIOS (0 a 1)
    W1 = np.random.rand(n_inputs, n_hidden)
    W2 = np.random.rand(n_hidden, n_outputs)

    errores = []
    epoch = 0
    inicio = time.time()

    while True:
        # Forward
        hidden = sigmoid(np.dot(X, W1))
        output = sigmoid(np.dot(hidden, W2))

        # Error
        error = Y - output
        mse = np.mean(error**2)
        errores.append(mse)

        # Criterio de parada
        if mse < epsilon or epoch >= max_epochs:
            break

        # Backpropagation
        d_output = error * sigmoid_derivative(output)
        d_hidden = np.dot(d_output, W2.T) * sigmoid_derivative(hidden)

        # Actualizar pesos
        W2 += eta * np.dot(hidden.T, d_output)
        W1 += eta * np.dot(X.T, d_hidden)

        epoch += 1

    tiempo = time.time() - inicio
    return W1, W2, errores, epoch, tiempo

# 4. Ejecutar 5 entrenamientos
resultados = []

for i in range(5):
    print(f"\n===== ENTRENAMIENTO {i+1} =====\n")
    W1, W2, errores, epocas, tiempo = entrenar_MLP(X, Y)

    resultados.append((W1, W2, errores, epocas, tiempo))

    print(f"Épocas: {epocas}")
    print(f"MSE final: {errores[-1]:.10f}")
    print(f"Tiempo: {tiempo:.4f} segundos")



===== ENTRENAMIENTO 1 =====

Épocas: 100000
MSE final: 0.0402713258
Tiempo: 5.5888 segundos

===== ENTRENAMIENTO 2 =====

Épocas: 100000
MSE final: 0.0313510356
Tiempo: 5.6174 segundos

===== ENTRENAMIENTO 3 =====

Épocas: 100000
MSE final: 0.0311199643
Tiempo: 6.0318 segundos

===== ENTRENAMIENTO 4 =====

Épocas: 100000
MSE final: 0.0325815218
Tiempo: 7.2109 segundos

===== ENTRENAMIENTO 5 =====

Épocas: 100000
MSE final: 0.0314553515
Tiempo: 6.1149 segundos


## Punto 2:
Dado que el problema se configura como un proceso típico de clasificación de patrones, implemente la rutina que posprocesa las salidas de la red (números reales) en enteros. Utilice el criterio de redondeo simétrico, es decir:

$$
y_i^{\text{pós}} =
\begin{cases}
1, & \text{si } y_i \ge 0.5 \\
0, & \text{si } y_i < 0.5
\end{cases}
$$

Úselo solo en el posprocesamiento del conjunto de prueba.


## Punto 3:
Para cada una de las cinco sesiones de entrenamiento, realice la validación utilizando el conjunto de prueba proporcionado en la tabla a continuación. Indique el porcentaje de precisión entre los valores deseados y los valores proporcionados por la red (tras el posprocesamiento) para todas las muestras de prueba.

| Muestra | x₁     | x₂     | x₃     | x₄     | d₁ | d₂ | d₃ | y₁ₚₒₛ | y₂ₚₒₛ | y₃ₚₒₛ | y₁ | y₂ | y₃ |
|---------|--------|--------|--------|--------|----|----|----|--------|--------|--------|----|----|----|
| 1       | 0.8622 | 0.7101 | 0.6236 | 0.7894 | 0  | 0  | 1  |        |        |        |    |    |    |
| 2       | 0.2741 | 0.1552 | 0.1333 | 0.1516 | 1  | 0  | 0  |        |        |        |    |    |    |
| 3       | 0.6772 | 0.8516 | 0.6543 | 0.7573 | 0  | 0  | 1  |        |        |        |    |    |    |
...


## Punto 4:
Explique la razón para realizar cinco sesiones de entrenamiento para la misma configuración topológica de una red MLP.


## Punto 5:
Para la mejor de las cinco sesiones de entrenamiento realizadas anteriormente, trace el gráfico respectivo de los valores del **error cuadrático medio (MSE)** en función de cada época de entrenamiento.
